<a href="https://colab.research.google.com/github/amr9/Langchain-big-query/blob/main/langchain%20big-query%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain

In [ ]:
!pip install openai
!pip install google-cloud-bigquery
!pip install jinja2

In [ ]:
!pip install chromadb
!pip install tiktoken

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""

In [ ]:
import sys
from langchain import PromptTemplate, OpenAI, LLMChain
from google.cloud import bigquery
import json
import argparse
from google.cloud import bigquery

In [ ]:
TEMPLATE = '''
Write a BigQuery SQL that achieves the following.
```
{{ content }}
```
The format of the target tables is as follows.
```json
{{ schema }}
```
Output the SQL in raw text.
    '''

In [ ]:
def get_schema(table_name: str) -> str:
    client = bigquery.Client()
    table = client.get_table(table_name)
    project_id = table.project
    dataset_id = table.dataset_id
    table_id = table.table_id
    schema = list(map(lambda x: x.to_api_repr(), table.schema))
    return {'project':project_id,'dataset':dataset_id,'table':table_id,'schema':schema}


In [ ]:
def get_schemas(table_names: list[str]):
    return json.dumps([get_schema(n) for n in table_names])

In [ ]:
def predict(content: str, table_names: list[str], verbose: bool = False):
    prompt = PromptTemplate(
        input_variables=["content","schema"],
        template=TEMPLATE,
        template_format='jinja2',
    )
    llm_chain = LLMChain(
        llm=OpenAI(temperature=0,model_name="gpt-3.5-turbo"), 
        prompt=prompt, 
        verbose=verbose,
    )
    return llm_chain.predict(content=content, schema=get_schemas(table_names))

In [ ]:
print(predict('print everything from the table',['symmetric-span-384617.langchain_test.people','symmetric-span-384617.langchain_test.telephones']))

In [ ]:
from google.cloud import bigquery


client = bigquery.Client()

query = """
SELECT *
FROM `table_name`
WHERE Freehold_building_size__ft2 > 6000
LIMIT 10
"""
results = client.query(query)

for row in results:
    print(row)

In [ ]:
from google.cloud import bigquery

# Create a BigQuery client object
client = bigquery.Client()

# List all tables in a dataset
dataset_ref = client.dataset('Final_New')
tables = client.list_tables(dataset_ref)

# Print the table names
for table in tables:   
    s=client.get_table(table)
    fields =s.schema
    print("\n"+table.table_id+"\n")
    for fields in fields:
      print(fields.name)



In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("/content/test.txt")
loaded_documents = loader.load()

In [ ]:
print(len(loaded_documents))

1


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(loaded_documents)

In [ ]:
print(len(documents))

6


In [ ]:
documents[5]

Document(page_content='the prciing cost 1000usd for project and we are a big company specialized in software projects', metadata={'source': '/content/test.txt'})

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
template = """
you are an agent that work in a company which have data of the projects that the company did with the technologies that they used ,
i want you to find if the following text that i will give you now is similiar like the projects we did if yes you tell him about the project and the technologies that we used ,
pricing of the company and small brief about it. the prciing cost 1000usd for project and we are a big company specialized in software projects,
if no you tell him about the company and the pricing : the cost of project is about 500 usd and we use alot of technologies . and here is the text that you search about:

Text:{query}

if you can't answer the question tell him about the company and the pricing of it
Context: the prciing cost 1000usd for project and we are a big company specialized in software projects


"""

In [ ]:
from langchain import PromptTemplate

In [ ]:
prompt = PromptTemplate(
    input_variables=["query"],
    template=template,
)

In [ ]:
print(prompt.format(query="we want to do action detection app the technologies that we may use language like python and machine learning library like tensorflow"))


you are an agent that work in a company which have data of the projects that the company did with the technologies that they used ,
i want you to find if the following text that i will give you now is similiar like the projects we did if yes you tell him about the project and the technologies that we used ,
pricing of the company and small brief about it. the prciing cost 1000usd for project and we are a big company specialized in software projects,
if no you tell him about the company and the pricing : the cost of project is about 500 usd and we use alot of technologies . and here is the text that you search about:

Text:we want to do action detection app the technologies that we may use language like python and machine learning library like tensorflow

if you can't answer the question tell him about the company and the pricing of it
Context: the prciing cost 1000usd for project and we are a big company specialized in software projects






In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
query = "we want to do action detection app"
queryWithTemplate = "you are an agent that work in a company which have data of the projects that the company did with the technologies that they used , i want you to find if the following text that i will give you now is similiar like the projects we did if yes you tell him about the project and the technologies that we used , pricing of the company and small brief about it. the prciing cost 1000usd for project and we are a big company specialized in software projects, if no you tell him about the company and the pricing : the cost of project is about 500 usd and we use alot of technologies . and here is the text that you search about:" + query

In [ ]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0,model_name="gpt-3.5-turbo"), vectorstore.as_retriever(), memory=memory)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:687: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
result = qa({"question": prompt.format(query="""I am first on here and  I need my chatbot.
I would like a chatbot that uses chatgpt and manychat to be on my business website to communicate with customers, I would like it to have data learning so i can upload content on what i do to teach the bot how to respond better.  Additionally i would like to to have integrations into my email and be able to respond to simple monthly updates.  I would like social media integration or a push to the website to interact with the bot. and ultimately. I have 4 businesses i want to be able to put it on as many businesses as i want. since i am using my credentials for gpt and many chat.""")})

In [ ]:
result['answer']

'No, I do not have experience developing chatbots using ChatGPT and ManyChat with those specific capabilities.'

In [ ]:
from google.cloud import bigquery
import json

def get_all_table_schemas():
    # create a BigQuery client object
    client = bigquery.Client()

    # list all datasets in the project
    datasets = client.list_datasets()

    # loop through each dataset and list all tables
    tables_schemas = {}
    for dataset in datasets:
        dataset_tables_schemas = {}
        dataset_ref = client.dataset(dataset.dataset_id)
        tables = client.list_tables(dataset_ref)
        for table in tables:
            table_ref = dataset_ref.table(table.table_id)
            table = client.get_table(table_ref)
            schema = list(map(lambda x: x.to_api_repr(), table.schema))
            dataset_tables_schemas[table.table_id] = schema
        tables_schemas[f"Project: {table.project} | Dataset: {dataset.dataset_id}"] = dataset_tables_schemas

    return json.dumps(tables_schemas, indent=2)

print(get_all_table_schemas())
